<a href="https://colab.research.google.com/github/niikun/langchain_tutorial/blob/main/Build_a_Simple_LLM_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Langchain  
---
## Build a Simple LLM Application  
### Get started
https://python.langchain.com/v0.2/docs/tutorials/llm_chain/


In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

# Detailed walkthrough
## Using Openai

In [15]:
!pip install openai
!pip install langchain
!pip install langchain-community
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.5 MB/s eta 0:00:00


In [16]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [17]:
from langchain_community import chat_models
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="Translate the following from English into Japanese"),
    HumanMessage(content="I got hungry.")
]

model.invoke(messages)

AIMessage(content='お腹が空きました。', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 22, 'total_tokens': 29}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-21354085-3b7d-4182-b7c1-d81d34ea1032-0')

In [25]:
AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4o', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc5d7c88-9615-48ab-a3c7-425232b562c5-0')

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4o', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc5d7c88-9615-48ab-a3c7-425232b562c5-0')

#Output Parsers  
モデルからのレスポンスがAIMessageであることに注目してください。  
これは文字列レスポンスとレスポンスに関する他のメタデータを含みます。文字列レスポンスだけを扱いたいこともよくあります。  
単純な出力パーサーを使うことで、このレスポンスだけを解析することができます。

In [27]:
!pip install langchain-core

In [29]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [30]:
result = model.invoke(messages)
parser.invoke(result)

'お腹が空きました。'

## chain

In [31]:
chain = model | parser

In [32]:
chain.invoke(messages)

'お腹が空きました。'

# Prompt Templates  
私たちはメッセージのリストを直接言語モデルに渡しています。このメッセージのリストはどこから来るのでしょうか？通常は、ユーザー入力とアプリケーションロジックの組み合わせから構築されます。  
このアプリケーションロジックは通常、ユーザー入力を受け取り、言語モデルに渡すためのメッセージリストに変換します。一般的な変換には、システムメッセージの追加や、ユーザー入力によるテンプレートのフォーマットなどがあります。  
PromptTemplatesはLangChainのコンセプトで、この変換を支援するために設計されています。生のユーザ入力を受け取り、言語モデルに渡す準備ができているデータ（プロンプト）を返します。

In [33]:
from langchain_core.prompts import ChatPromptTemplate

In [35]:
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [36]:
result = prompt_template.invoke({"language": "italian", "text": "I got hungry."})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='I got hungry.')])

In [37]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='I got hungry.')]

In [38]:
chain = prompt_template | model | parser

In [39]:
chain.invoke({"language":"Japanese","text":"I got hungry!"})

'お腹が空いた！'

# Serving with LangServe  
さて、アプリケーションを構築したら、それを提供する必要があります。そこでLangServeの登場です。  
LangServeは開発者がLangChainチェーンをREST APIとしてデプロイするのを助けます。  
LangChainを使うのにLangServeを使う必要はありませんが、このガイドではLangServeを使ってアプリをデプロイする方法を紹介します。
